# 01. Bronze Layer: ダミーデータ生成 (SQL版)

## 概要
Paper08（関節リウマチの有病率と治療動向）の再現に必要なダミーデータをSparkSQLで生成します。

## 生成するデータ
1. **レセプト基本データ (RE)** - 患者基本情報
2. **傷病データ (SY)** - 診断コード（ICD-10）
3. **医薬品データ (IY)** - 処方薬情報
4. **診療行為データ (SI)** - 手術・検査情報
5. **保険者データ (HO)** - 保険者情報

## データ設計方針
- 論文のDefinition 3に基づくRA患者を含むように設計
- 年齢分布は論文のTable 2に近似
- 薬剤使用パターンは論文のTable 3に近似

## 実行環境
- Databricks (SparkSQL)

## 1. 定数・マスタデータの定義

In [ ]:
-- 既存のテーブルがあれば削除
DROP TABLE IF EXISTS bronze_patients;
DROP TABLE IF EXISTS bronze_re_receipt;
DROP TABLE IF EXISTS bronze_sy_disease;
DROP TABLE IF EXISTS bronze_iy_medication;
DROP TABLE IF EXISTS bronze_si_procedure;
DROP TABLE IF EXISTS bronze_ho_insurer;

-- 一時テーブルも削除
DROP TABLE IF EXISTS tmp_age_groups;
DROP TABLE IF EXISTS tmp_ra_icd10_codes;
DROP TABLE IF EXISTS tmp_non_ra_icd10_codes;
DROP TABLE IF EXISTS tmp_dmards_master;
DROP TABLE IF EXISTS tmp_months;
DROP TABLE IF EXISTS tmp_patient_months;

In [ ]:
-- 年齢層マスタ（論文Table 2に基づくRA患者年齢分布）
CREATE OR REPLACE TEMPORARY VIEW tmp_age_groups AS
SELECT * FROM VALUES
    ('16-19', 16, 19, 0.005),
    ('20-29', 20, 29, 0.015),
    ('30-39', 30, 39, 0.048),
    ('40-49', 40, 49, 0.101),
    ('50-59', 50, 59, 0.149),
    ('60-69', 60, 69, 0.264),
    ('70-79', 70, 79, 0.286),
    ('80-84', 80, 84, 0.061),
    ('85+', 85, 100, 0.070)
AS t(age_group, min_age, max_age, ra_distribution);

In [ ]:
-- RA関連ICD-10コードマスタ
CREATE OR REPLACE TEMPORARY VIEW tmp_ra_icd10_codes AS
SELECT * FROM VALUES
    ('M050'), ('M051'), ('M052'), ('M053'), ('M058'), ('M059'),  -- 血清陽性RA
    ('M060'), ('M062'), ('M063'), ('M068'), ('M069'),            -- その他のRA
    ('M080'), ('M083'), ('M084'), ('M088'), ('M089')             -- 若年性関節炎
AS t(icd10_code);

In [ ]:
-- 非RA関連ICD-10コードマスタ（コントロール用）
CREATE OR REPLACE TEMPORARY VIEW tmp_non_ra_icd10_codes AS
SELECT * FROM VALUES
    ('J00'),   -- 急性鼻咽頭炎
    ('J06'),   -- 急性上気道感染症
    ('I10'),   -- 本態性高血圧
    ('E11'),   -- 2型糖尿病
    ('K21'),   -- 胃食道逆流症
    ('M54'),   -- 背部痛
    ('G43'),   -- 片頭痛
    ('F32')    -- うつ病エピソード
AS t(icd10_code);

In [ ]:
-- DMARDsマスタ
CREATE OR REPLACE TEMPORARY VIEW tmp_dmards_master AS
SELECT * FROM VALUES
    -- csDMARDs（従来型合成DMARD）
    ('MTX', '1199101', 'メトトレキサート', 'csDMARD', 0.634),
    ('MTX', '1199102', 'メトトレキサート', 'csDMARD', 0.634),
    ('SSZ', '1199201', 'サラゾスルファピリジン', 'csDMARD', 0.249),
    ('TAC', '1199301', 'タクロリムス', 'csDMARD', 0.106),
    ('BUC', '1199401', 'ブシラミン', 'csDMARD', 0.145),
    ('IGT', '1199501', 'イグラチモド', 'csDMARD', 0.047),
    ('LEF', '1199601', 'レフルノミド', 'csDMARD', 0.030),
    -- bDMARDs - TNF阻害薬
    ('IFX', '4400101', 'インフリキシマブ', 'TNFI', 0.027),
    ('ETN', '4400102', 'エタネルセプト', 'TNFI', 0.027),
    ('ADA', '4400103', 'アダリムマブ', 'TNFI', 0.027),
    ('GLM', '4400104', 'ゴリムマブ', 'TNFI', 0.027),
    ('CZP', '4400105', 'セルトリズマブペゴル', 'TNFI', 0.027),
    -- bDMARDs - IL-6阻害薬
    ('TCZ', '4400201', 'トシリズマブ', 'IL6I', 0.030),
    ('SAR', '4400202', 'サリルマブ', 'IL6I', 0.030),
    -- bDMARDs - アバタセプト
    ('ABT', '4400301', 'アバタセプト', 'ABT', 0.045),
    -- tsDMARDs - JAK阻害薬
    ('TOF', '4400401', 'トファシチニブ', 'JAKi', 0.005),
    ('BAR', '4400402', 'バリシチニブ', 'JAKi', 0.005),
    -- 経口ステロイド
    ('PSL', '2454001', 'プレドニゾロン', 'CS', 0.450),
    -- NSAIDs
    ('NSAID1', '1141001', 'ロキソプロフェン', 'NSAID', 0.300)
AS t(drug_abbrev, drug_code, drug_name, drug_category, usage_rate);

In [ ]:
-- 手術・検査マスタ
CREATE OR REPLACE TEMPORARY VIEW tmp_procedures_master AS
SELECT * FROM VALUES
    -- 手術
    ('TJR', 'K0821', '人工関節置換術', 'SURGERY', 0.0093),
    ('ARTHROPLASTY', 'K0801', '関節形成術', 'SURGERY', 0.0032),
    ('SYNOVECTOMY', 'K0661', '滑膜切除術', 'SURGERY', 0.0013),
    -- 検査
    ('ULTRASOUND', 'D2151', '関節超音波検査', 'IMAGING', 0.180),
    ('BMD', 'D2171', '骨密度測定', 'IMAGING', 0.100)
AS t(procedure_abbrev, procedure_code, procedure_name, procedure_type, usage_rate);

## 2. 患者基本情報の生成

In [ ]:
-- 患者基本情報の生成
-- 総患者数: 10,000人（論文の1/1000スケール）
-- RA有病率: 0.65%（約97人がRA候補）

CREATE OR REPLACE TABLE bronze_patients AS
WITH 
-- 連番生成（0-9999）
patient_ids AS (
    SELECT explode(sequence(0, 9999)) AS patient_id
),

-- RA候補かどうかを決定（最初の約150人をRA候補とする = Definition 0相当）
patients_with_ra_flag AS (
    SELECT 
        patient_id,
        CASE WHEN patient_id < 150 THEN true ELSE false END AS is_ra_candidate
    FROM patient_ids
),

-- 年齢層の累積分布を計算
age_group_cumulative AS (
    SELECT 
        age_group, min_age, max_age, ra_distribution,
        SUM(ra_distribution) OVER (ORDER BY min_age ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_prob
    FROM tmp_age_groups
),

-- RA候補患者の年齢割り当て
ra_patients AS (
    SELECT 
        p.patient_id,
        p.is_ra_candidate,
        -- 乱数で年齢層を選択
        (
            SELECT ag.age_group 
            FROM age_group_cumulative ag 
            WHERE ag.cumulative_prob >= (abs(hash(p.patient_id, 'age_group')) % 1000) / 1000.0
            ORDER BY ag.min_age
            LIMIT 1
        ) AS selected_age_group,
        -- 性別（女性76.3%）
        CASE WHEN (abs(hash(p.patient_id, 'sex')) % 1000) / 1000.0 < 0.763 THEN '2' ELSE '1' END AS sex
    FROM patients_with_ra_flag p
    WHERE p.is_ra_candidate = true
),

-- 非RA患者の年齢割り当て（一般人口分布）
non_ra_patients AS (
    SELECT 
        p.patient_id,
        p.is_ra_candidate,
        -- 16-89歳の一様分布
        CAST(NULL AS STRING) AS selected_age_group,
        -- 性別（女性51%）
        CASE WHEN (abs(hash(p.patient_id, 'sex')) % 1000) / 1000.0 < 0.51 THEN '2' ELSE '1' END AS sex
    FROM patients_with_ra_flag p
    WHERE p.is_ra_candidate = false
),

-- 全患者を結合
all_patients AS (
    SELECT * FROM ra_patients
    UNION ALL
    SELECT * FROM non_ra_patients
),

-- 年齢層から具体的な年齢を決定
patients_with_age AS (
    SELECT 
        p.patient_id,
        p.is_ra_candidate,
        p.sex,
        CASE 
            WHEN p.is_ra_candidate THEN
                COALESCE(
                    (
                        SELECT ag.min_age + (abs(hash(p.patient_id, 'age_detail')) % (ag.max_age - ag.min_age + 1))
                        FROM tmp_age_groups ag
                        WHERE ag.age_group = p.selected_age_group
                    ),
                    50  -- デフォルト
                )
            ELSE
                16 + (abs(hash(p.patient_id, 'age')) % 74)  -- 16-89歳
        END AS age
    FROM all_patients p
)

-- 最終的な患者テーブル
SELECT 
    patient_id,
    sha2(concat('patient_', cast(patient_id as string)), 256) AS common_key,
    age,
    CASE 
        WHEN age BETWEEN 16 AND 19 THEN '16-19'
        WHEN age BETWEEN 20 AND 29 THEN '20-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        WHEN age BETWEEN 50 AND 59 THEN '50-59'
        WHEN age BETWEEN 60 AND 69 THEN '60-69'
        WHEN age BETWEEN 70 AND 79 THEN '70-79'
        WHEN age BETWEEN 80 AND 84 THEN '80-84'
        ELSE '85+'
    END AS age_group,
    sex,
    date_sub(
        date('2017-10-01'),
        age * 365 + (abs(hash(patient_id, 'birth_day')) % 365)
    ) AS birth_date,
    is_ra_candidate
FROM patients_with_age;

In [ ]:
-- 患者データの確認
SELECT 
    '総患者数' AS metric,
    COUNT(*) AS value
FROM bronze_patients
UNION ALL
SELECT 
    'RA候補患者数' AS metric,
    COUNT(*) AS value
FROM bronze_patients WHERE is_ra_candidate = true
UNION ALL
SELECT 
    'RA候補女性比率(%)' AS metric,
    ROUND(AVG(CASE WHEN sex = '2' THEN 100.0 ELSE 0.0 END), 1) AS value
FROM bronze_patients WHERE is_ra_candidate = true;

In [ ]:
-- RA候補患者の年齢分布
SELECT 
    age_group,
    COUNT(*) AS patient_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) AS percentage
FROM bronze_patients
WHERE is_ra_candidate = true
GROUP BY age_group
ORDER BY 
    CASE age_group
        WHEN '16-19' THEN 1
        WHEN '20-29' THEN 2
        WHEN '30-39' THEN 3
        WHEN '40-49' THEN 4
        WHEN '50-59' THEN 5
        WHEN '60-69' THEN 6
        WHEN '70-79' THEN 7
        WHEN '80-84' THEN 8
        WHEN '85+' THEN 9
    END;

## 3. レセプト基本データ (RE) の生成

In [ ]:
-- 診療月マスタ（2017年4月〜2018年3月）
CREATE OR REPLACE TEMPORARY VIEW tmp_months AS
SELECT explode(sequence(date('2017-04-01'), date('2018-03-01'), interval 1 month)) AS visit_month;

In [ ]:
-- レセプト基本データの生成
CREATE OR REPLACE TABLE bronze_re_receipt AS
WITH 
-- 患者ごとの受診月数を決定
patient_visit_counts AS (
    SELECT 
        patient_id,
        common_key,
        sex,
        birth_date,
        is_ra_candidate,
        CASE 
            WHEN is_ra_candidate THEN 2 + (abs(hash(patient_id, 'visit_count')) % 11)  -- 2-12ヶ月
            ELSE 1 + (abs(hash(patient_id, 'visit_count')) % 4)  -- 1-4ヶ月
        END AS n_visits
    FROM bronze_patients
),

-- 患者と月のクロス結合
patient_months AS (
    SELECT 
        p.patient_id,
        p.common_key,
        p.sex,
        p.birth_date,
        p.is_ra_candidate,
        p.n_visits,
        m.visit_month,
        ROW_NUMBER() OVER (PARTITION BY p.patient_id ORDER BY abs(hash(p.patient_id, date_format(m.visit_month, 'yyyyMM')))) AS month_rank
    FROM patient_visit_counts p
    CROSS JOIN tmp_months m
),

-- 実際に受診する月を選択
selected_visits AS (
    SELECT *
    FROM patient_months
    WHERE month_rank <= n_visits
)

-- レセプトレコードを生成
SELECT 
    common_key AS `共通キー`,
    concat('RCP', lpad(cast(patient_id as string), 8, '0'), date_format(visit_month, 'yyyyMM')) AS `検索番号`,
    '1' AS `データ識別`,
    '1' AS `行番号`,
    '0' AS `枝番号`,
    'RE' AS `レコード識別情報`,
    cast(1000000 + (abs(hash(patient_id, date_format(visit_month, 'yyyyMM'), 'receipt_no')) % 9000000) as string) AS `レセプト番号`,
    '1112' AS `レセプト種別`,
    date_format(visit_month, 'yyyyMM') AS `診療年月`,
    sex AS `男女区分`,
    date_format(birth_date, 'yyyy-MM-dd') AS `生年月日`,
    CASE WHEN month(visit_month) >= 4 THEN '2017' ELSE '2017' END AS fy,
    date_format(visit_month, 'yyyy') AS year,
    date_format(visit_month, 'MM') AS month,
    lpad(cast(1 + (abs(hash(patient_id, 'pref')) % 47) as string), 2, '0') AS prefecture_number
FROM selected_visits;

In [ ]:
-- レセプトデータの確認
SELECT 
    '総レセプト数' AS metric,
    COUNT(*) AS value
FROM bronze_re_receipt
UNION ALL
SELECT 
    'ユニーク患者数' AS metric,
    COUNT(DISTINCT `共通キー`) AS value
FROM bronze_re_receipt;

In [ ]:
-- 診療年月別レセプト数
SELECT 
    `診療年月`,
    COUNT(*) AS receipt_count
FROM bronze_re_receipt
GROUP BY `診療年月`
ORDER BY `診療年月`;

## 4. 傷病データ (SY) の生成

In [ ]:
-- 傷病データの生成
CREATE OR REPLACE TABLE bronze_sy_disease AS
WITH 
-- レセプトと患者情報を結合
receipt_with_patient AS (
    SELECT 
        r.*,
        p.is_ra_candidate,
        p.patient_id
    FROM bronze_re_receipt r
    JOIN bronze_patients p ON r.`共通キー` = p.common_key
),

-- 傷病名数を決定（1-4件）
receipt_disease_counts AS (
    SELECT 
        *,
        1 + (abs(hash(`検索番号`, 'disease_count')) % 4) AS n_diseases
    FROM receipt_with_patient
),

-- 傷病名番号を展開
disease_numbers AS (
    SELECT 
        r.*,
        d.disease_num
    FROM receipt_disease_counts r
    LATERAL VIEW explode(sequence(1, r.n_diseases)) d AS disease_num
),

-- RA関連ICD10コードの配列を取得
ra_codes AS (
    SELECT collect_list(icd10_code) AS codes FROM tmp_ra_icd10_codes
),

-- 非RA関連ICD10コードの配列を取得
non_ra_codes AS (
    SELECT collect_list(icd10_code) AS codes FROM tmp_non_ra_icd10_codes
)

-- 傷病レコードを生成
SELECT 
    d.`共通キー`,
    d.`検索番号`,
    d.`データ識別`,
    cast(d.disease_num as string) AS `行番号`,
    '0' AS `枝番号`,
    'SY' AS `レコード識別情報`,
    CASE 
        WHEN d.is_ra_candidate AND d.disease_num = 1 THEN
            concat('SY', ra.codes[abs(hash(d.`検索番号`, 'ra_code')) % size(ra.codes)])
        ELSE
            concat('SY', nra.codes[abs(hash(d.`検索番号`, d.disease_num, 'non_ra_code')) % size(nra.codes)])
    END AS `傷病名コード`,
    concat(d.`診療年月`, lpad(cast(1 + (abs(hash(d.`検索番号`, d.disease_num, 'start_day')) % 28) as string), 2, '0')) AS `診療開始日`,
    cast(1 + (abs(hash(d.`検索番号`, d.disease_num, 'outcome')) % 3) as string) AS `転帰区分`,
    '' AS `修飾語コード`,
    CASE 
        WHEN d.is_ra_candidate AND d.disease_num = 1 THEN
            concat('傷病名_', ra.codes[abs(hash(d.`検索番号`, 'ra_code')) % size(ra.codes)])
        ELSE
            concat('傷病名_', nra.codes[abs(hash(d.`検索番号`, d.disease_num, 'non_ra_code')) % size(nra.codes)])
    END AS `傷病名`,
    CASE 
        WHEN d.is_ra_candidate AND d.disease_num = 1 THEN
            ra.codes[abs(hash(d.`検索番号`, 'ra_code')) % size(ra.codes)]
        ELSE
            nra.codes[abs(hash(d.`検索番号`, d.disease_num, 'non_ra_code')) % size(nra.codes)]
    END AS `ICD10コード`,
    d.fy,
    d.year,
    d.month,
    d.prefecture_number
FROM disease_numbers d
CROSS JOIN ra_codes ra
CROSS JOIN non_ra_codes nra;

In [ ]:
-- 傷病データの確認
SELECT 
    '総傷病レコード数' AS metric,
    COUNT(*) AS value
FROM bronze_sy_disease
UNION ALL
SELECT 
    'RA関連ICD10レコード数' AS metric,
    COUNT(*) AS value
FROM bronze_sy_disease
WHERE `ICD10コード` IN (SELECT icd10_code FROM tmp_ra_icd10_codes);

In [ ]:
-- ICD-10コード分布
SELECT 
    `ICD10コード`,
    COUNT(*) AS count
FROM bronze_sy_disease
GROUP BY `ICD10コード`
ORDER BY count DESC
LIMIT 15;

## 5. 医薬品データ (IY) の生成

In [ ]:
-- 医薬品データの生成
CREATE OR REPLACE TABLE bronze_iy_medication AS
WITH 
-- レセプトと患者情報を結合
receipt_with_patient AS (
    SELECT 
        r.*,
        p.is_ra_candidate,
        p.patient_id,
        p.age
    FROM bronze_re_receipt r
    JOIN bronze_patients p ON r.`共通キー` = p.common_key
),

-- DMARDsマスタを取得（薬剤ごとにユニーク）
dmards_unique AS (
    SELECT 
        drug_abbrev,
        MIN(drug_code) AS drug_code,
        MIN(drug_name) AS drug_name,
        MIN(drug_category) AS drug_category,
        MIN(usage_rate) AS usage_rate
    FROM tmp_dmards_master
    GROUP BY drug_abbrev
),

-- 各レセプトに対して各薬剤の処方可否を決定
receipt_drug_decisions AS (
    SELECT 
        r.`共通キー`,
        r.`検索番号`,
        r.`データ識別`,
        r.fy,
        r.year,
        r.month,
        r.prefecture_number,
        r.is_ra_candidate,
        r.patient_id,
        r.age,
        d.drug_abbrev,
        d.drug_code,
        d.drug_name,
        d.drug_category,
        -- 年齢による調整
        CASE 
            WHEN d.drug_abbrev = 'MTX' AND r.age >= 85 THEN d.usage_rate * 0.6
            WHEN d.drug_abbrev = 'MTX' AND r.age >= 80 THEN d.usage_rate * 0.8
            ELSE d.usage_rate
        END AS adjusted_usage_rate,
        -- 処方決定用の乱数
        (abs(hash(r.`検索番号`, d.drug_abbrev)) % 1000) / 1000.0 AS rand_val
    FROM receipt_with_patient r
    CROSS JOIN dmards_unique d
    WHERE r.is_ra_candidate = true
),

-- 実際に処方される薬剤を選択
prescribed_drugs AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY `検索番号` ORDER BY drug_abbrev) AS line_num
    FROM receipt_drug_decisions
    WHERE rand_val < adjusted_usage_rate
),

-- 非RA患者にも一般薬を追加（30%の確率）
non_ra_other_drugs AS (
    SELECT 
        r.`共通キー`,
        r.`検索番号`,
        r.`データ識別`,
        r.fy,
        r.year,
        r.month,
        r.prefecture_number,
        r.is_ra_candidate,
        r.patient_id,
        r.age,
        'OTHER' AS drug_abbrev,
        concat('OTHER', cast(1000 + (abs(hash(r.`検索番号`, 'other_drug')) % 9000) as string)) AS drug_code,
        '一般薬' AS drug_name,
        'OTHER' AS drug_category,
        1 AS line_num
    FROM receipt_with_patient r
    WHERE r.is_ra_candidate = false
      AND (abs(hash(r.`検索番号`, 'other_drug_decision')) % 1000) / 1000.0 < 0.3
)

-- 最終的な医薬品レコード
SELECT 
    `共通キー`,
    `検索番号`,
    `データ識別`,
    cast(line_num as string) AS `行番号`,
    '0' AS `枝番号`,
    'IY' AS `レコード識別情報`,
    '21' AS `診療識別`,
    '1' AS `負担区分`,
    drug_code AS `医薬品コード`,
    cast(1 + (abs(hash(`検索番号`, drug_abbrev, 'usage')) % 30) as string) AS `使用量`,
    cast(100 + (abs(hash(`検索番号`, drug_abbrev, 'points')) % 4900) as string) AS `点数`,
    '1' AS `回数`,
    drug_category,
    drug_abbrev AS drug_name,
    fy,
    year,
    month,
    prefecture_number
FROM prescribed_drugs

UNION ALL

SELECT 
    `共通キー`,
    `検索番号`,
    `データ識別`,
    cast(line_num as string) AS `行番号`,
    '0' AS `枝番号`,
    'IY' AS `レコード識別情報`,
    '21' AS `診療識別`,
    '1' AS `負担区分`,
    drug_code AS `医薬品コード`,
    cast(1 + (abs(hash(`検索番号`, 'other_usage')) % 30) as string) AS `使用量`,
    cast(10 + (abs(hash(`検索番号`, 'other_points')) % 490) as string) AS `点数`,
    '1' AS `回数`,
    drug_category,
    drug_abbrev AS drug_name,
    fy,
    year,
    month,
    prefecture_number
FROM non_ra_other_drugs;

In [ ]:
-- 医薬品データの確認
SELECT 
    '総医薬品レコード数' AS metric,
    COUNT(*) AS value
FROM bronze_iy_medication;

In [ ]:
-- 薬剤カテゴリ別分布
SELECT 
    drug_category,
    COUNT(*) AS count
FROM bronze_iy_medication
GROUP BY drug_category
ORDER BY count DESC;

## 6. 診療行為データ (SI) の生成

In [ ]:
-- 診療行為データの生成
CREATE OR REPLACE TABLE bronze_si_procedure AS
WITH 
-- レセプトと患者情報を結合
receipt_with_patient AS (
    SELECT 
        r.*,
        p.is_ra_candidate,
        p.patient_id,
        p.age
    FROM bronze_re_receipt r
    JOIN bronze_patients p ON r.`共通キー` = p.common_key
),

-- RA患者の手術・検査を決定
ra_procedures AS (
    SELECT 
        r.`共通キー`,
        r.`検索番号`,
        r.`データ識別`,
        r.fy,
        r.year,
        r.month,
        r.prefecture_number,
        r.age,
        pm.procedure_abbrev,
        pm.procedure_code,
        pm.procedure_name,
        pm.procedure_type,
        -- 年齢による調整
        CASE 
            WHEN pm.procedure_abbrev = 'TJR' AND r.age BETWEEN 70 AND 84 THEN pm.usage_rate * 1.5
            WHEN pm.procedure_abbrev = 'SYNOVECTOMY' AND r.age < 70 THEN pm.usage_rate * 1.5
            WHEN pm.procedure_abbrev = 'BMD' THEN 0.05 + (r.age / 100.0) * 0.15
            ELSE pm.usage_rate
        END AS adjusted_rate,
        (abs(hash(r.`検索番号`, pm.procedure_abbrev)) % 1000) / 1000.0 AS rand_val
    FROM receipt_with_patient r
    CROSS JOIN tmp_procedures_master pm
    WHERE r.is_ra_candidate = true
),

-- 実施される手術・検査を選択
performed_procedures AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY `検索番号` ORDER BY procedure_abbrev) AS line_num
    FROM ra_procedures
    WHERE rand_val < adjusted_rate
),

-- 基本診療行為（全患者）
basic_visits AS (
    SELECT 
        `共通キー`,
        `検索番号`,
        `データ識別`,
        fy,
        year,
        month,
        prefecture_number,
        'VISIT' AS procedure_abbrev,
        '111000110' AS procedure_code,
        '初診料' AS procedure_name,
        'VISIT' AS procedure_type,
        -- 行番号は他の診療行為の後
        COALESCE(
            (SELECT MAX(line_num) + 1 FROM performed_procedures pp WHERE pp.`検索番号` = r.`検索番号`),
            1
        ) AS line_num
    FROM bronze_re_receipt r
)

-- 最終的な診療行為レコード
SELECT 
    `共通キー`,
    `検索番号`,
    `データ識別`,
    cast(line_num as string) AS `行番号`,
    '0' AS `枝番号`,
    'SI' AS `レコード識別情報`,
    CASE 
        WHEN procedure_type = 'SURGERY' THEN '50'
        WHEN procedure_type = 'IMAGING' THEN '60'
        ELSE '11'
    END AS `診療識別`,
    '1' AS `負担区分`,
    procedure_code AS `診療行為コード`,
    '1' AS `数量データ`,
    CASE 
        WHEN procedure_type = 'SURGERY' THEN cast(10000 + (abs(hash(`検索番号`, procedure_abbrev, 'points')) % 40000) as string)
        WHEN procedure_type = 'IMAGING' THEN cast(100 + (abs(hash(`検索番号`, procedure_abbrev, 'points')) % 900) as string)
        ELSE cast(100 + (abs(hash(`検索番号`, 'visit_points')) % 200) as string)
    END AS `点数`,
    '1' AS `回数`,
    procedure_type,
    procedure_name,
    fy,
    year,
    month,
    prefecture_number
FROM performed_procedures

UNION ALL

SELECT 
    `共通キー`,
    `検索番号`,
    `データ識別`,
    cast(line_num as string) AS `行番号`,
    '0' AS `枝番号`,
    'SI' AS `レコード識別情報`,
    '11' AS `診療識別`,
    '1' AS `負担区分`,
    procedure_code AS `診療行為コード`,
    '1' AS `数量データ`,
    cast(100 + (abs(hash(`検索番号`, 'visit_points')) % 200) as string) AS `点数`,
    '1' AS `回数`,
    procedure_type,
    procedure_name,
    fy,
    year,
    month,
    prefecture_number
FROM basic_visits;

In [ ]:
-- 診療行為データの確認
SELECT 
    '総診療行為レコード数' AS metric,
    COUNT(*) AS value
FROM bronze_si_procedure;

In [ ]:
-- 診療行為タイプ別分布
SELECT 
    procedure_type,
    COUNT(*) AS count
FROM bronze_si_procedure
GROUP BY procedure_type
ORDER BY count DESC;

## 7. 保険者データ (HO) の生成

In [ ]:
-- 保険者データの生成
CREATE OR REPLACE TABLE bronze_ho_insurer AS
SELECT 
    `共通キー`,
    `検索番号`,
    `データ識別`,
    '1' AS `行番号`,
    '0' AS `枝番号`,
    'HO' AS `レコード識別情報`,
    cast(10000000 + (abs(hash(`検索番号`, 'insurer_no')) % 90000000) as string) AS `保険者番号`,
    concat('SYM', cast(1000 + (abs(hash(`検索番号`, 'symbol')) % 9000) as string)) AS `被保険者証等記号`,
    cast(100000 + (abs(hash(`検索番号`, 'insured_no')) % 900000) as string) AS `被保険者証等番号`,
    cast(1 + (abs(hash(`検索番号`, 'visit_days')) % 15) as string) AS `診療実日数`,
    cast(100 + (abs(hash(`検索番号`, 'total_points')) % 9900) as string) AS `合計点数`,
    fy,
    year,
    month,
    prefecture_number
FROM bronze_re_receipt;

In [ ]:
-- 保険者データの確認
SELECT 
    '総保険者レコード数' AS metric,
    COUNT(*) AS value
FROM bronze_ho_insurer;

## 8. データ品質サマリー

In [ ]:
-- Bronze Layer 全体サマリー
SELECT 'bronze_patients' AS table_name, COUNT(*) AS record_count FROM bronze_patients
UNION ALL
SELECT 'bronze_re_receipt', COUNT(*) FROM bronze_re_receipt
UNION ALL
SELECT 'bronze_sy_disease', COUNT(*) FROM bronze_sy_disease
UNION ALL
SELECT 'bronze_iy_medication', COUNT(*) FROM bronze_iy_medication
UNION ALL
SELECT 'bronze_si_procedure', COUNT(*) FROM bronze_si_procedure
UNION ALL
SELECT 'bronze_ho_insurer', COUNT(*) FROM bronze_ho_insurer;

In [ ]:
-- RA候補患者の統計
SELECT 
    '総患者数' AS metric,
    COUNT(*) AS value
FROM bronze_patients
UNION ALL
SELECT 
    'RA候補患者数',
    COUNT(*)
FROM bronze_patients WHERE is_ra_candidate = true
UNION ALL
SELECT 
    'RA候補女性比率(%)',
    ROUND(AVG(CASE WHEN sex = '2' THEN 100.0 ELSE 0.0 END), 1)
FROM bronze_patients WHERE is_ra_candidate = true
UNION ALL
SELECT 
    'RA候補平均年齢',
    ROUND(AVG(age), 1)
FROM bronze_patients WHERE is_ra_candidate = true;

In [ ]:
-- RA候補患者の年齢分布と論文値との比較
WITH actual_dist AS (
    SELECT 
        age_group,
        COUNT(*) AS patient_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) AS actual_pct
    FROM bronze_patients
    WHERE is_ra_candidate = true
    GROUP BY age_group
)
SELECT 
    a.age_group,
    a.patient_count,
    a.actual_pct,
    ROUND(t.ra_distribution * 100, 1) AS paper_pct
FROM actual_dist a
JOIN tmp_age_groups t ON a.age_group = t.age_group
ORDER BY t.min_age;

In [ ]:
-- RA関連薬剤の処方率
WITH ra_receipts AS (
    SELECT DISTINCT r.`検索番号`
    FROM bronze_re_receipt r
    JOIN bronze_patients p ON r.`共通キー` = p.common_key
    WHERE p.is_ra_candidate = true
),
drug_usage AS (
    SELECT 
        m.drug_category,
        COUNT(DISTINCT m.`検索番号`) AS prescribed_count
    FROM bronze_iy_medication m
    JOIN ra_receipts rr ON m.`検索番号` = rr.`検索番号`
    WHERE m.drug_category != 'OTHER'
    GROUP BY m.drug_category
)
SELECT 
    d.drug_category,
    d.prescribed_count,
    (SELECT COUNT(*) FROM ra_receipts) AS total_ra_receipts,
    ROUND(d.prescribed_count * 100.0 / (SELECT COUNT(*) FROM ra_receipts), 1) AS usage_rate_pct
FROM drug_usage d
ORDER BY d.prescribed_count DESC;

## 9. データの永続化（オプション）

Databricksでは、上記で作成したテーブルは既にDelta Lakeテーブルとして永続化されています。
必要に応じて、特定のカタログ・スキーマに保存することも可能です。

In [ ]:
-- 生成完了メッセージ
SELECT 
    '========================================' AS message
UNION ALL
SELECT 'Bronze Layer 生成完了'
UNION ALL
SELECT '========================================';